In [1]:
#%%
import pandas as pd
import numpy as np
import xlwings as xw
import os
from pathlib import Path
import hide

In [3]:
# Read and initialize values
file = 'J12209 MODEC - EQUINOR BACALHAU FPSO - COM R7.xlsm'
downloads = '/Users/infowizard/Downloads/'
file_path = Path(downloads, file)
wb = xw.Book(file_path, password=hide.password)
visible_sheets = [sht.name for sht in wb.sheets if sht.visible]
# Read and set currency from FX sheet
fx = wb.sheets['FX']
exchange_rates = dict(fx.range('A2:B9')).value
quoted_currency = fx.range('B12').value
project_info = dict(fx.range('A36:B45').value)

In [96]:
# Read system sheets
cols = ['No', 'Qty', 'Unit', 'Description', 'Unit Price', 'Subtotal', 'Part', 'Cur', 'Cost', 'SubCost', 'Discount']
systems = pd.DataFrame()
defaults = {}
skip_sheets = ['FX', 'Cover', 'Intro', 'ES', 'T&C']
for sheet in visible_sheets:
    if sheet not in skip_sheets:
        ws = wb.sheets[sheet]
        escalation = dict(ws.range('K2:L5').value)
        default_mu = ws.range('H5')
        escalation['default_mu'] = default_mu
        defaults[sheet] = escalation
        last_row = ws.range('D100000').end('up').row  #Returns a number
        last_cell = 'K' + str(last_row) 
        data = ws.range('A8:' + last_cell).options(pd.DataFrame, index=False).value
        data.columns = cols
        data['System'] = str(sheet)
        data['Category'] = 'Product'
        systems = pd.concat([systems, data], join='outer')

In [113]:
# Read Engineering Services
es_cols = ['No', 'Qty', 'Unit', 'Description', 'Unit Price', 'Subtotal', 'Part', 'Cur', 'Cost', 'Man-days', 'Subcost']
es = wb.sheets['ES']
es_last_row = es.range('D100000').end('up').row
es_last_cell = 'K' + str(es_last_row)
eng_service = es.range('A8:' + es_last_cell).options(pd.DataFrame, index=False).value
eng_service.columns = es_cols